Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [ ]:
#loading the data from the csv file to pandas dataframe
try:
    movies_data = pd.read_csv('/content/movies.csv', engine='python', on_bad_lines='skip')
except Exception as e:
    print(f"An error occurred: {e}")
    # If skipping bad lines doesn't work, try reading with a different delimiter or quoting
    try:
        movies_data = pd.read_csv('/content/movies.csv', engine='python', sep=',', quotechar='"', on_bad_lines='skip')
    except Exception as e2:
        print(f"A second attempt with different parameters failed: {e2}")
        # As a last resort, try reading line by line
        try:
            data = []
            with open('/content/movies.csv', 'r', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    if i < 500: # Limit the number of lines to read to avoid excessive memory usage
                        try:
                            data.append(line.strip().split(',')) # Basic splitting, may need adjustment based on actual delimiter
                        except Exception as line_e:
                            print(f"Error reading line {i}: {line_e}")
                            continue
            # Attempt to create a DataFrame from the manually read data
            # This will likely require further processing to handle headers and column types correctly
            movies_data = pd.DataFrame(data)
            print("Successfully read some data manually, but further processing is needed.")
        except Exception as e3:
            print(f"Manual reading also failed: {e3}")

In [ ]:
#printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of rows and columns in the dataframe
movies_data.shape

(533, 24)

In [ ]:
#selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
#replacing the missing values with null values

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
#combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [ ]:
print(combined_features)

0      Action Adventure Fantasy Science Fiction cultu...
1      Adventure Fantasy Action ocean drug abuse exot...
2      Action Adventure Crime spy based on novel secr...
3      Action Crime Drama Thriller dc comics crime fi...
4      Action Adventure Science Fiction based on nove...
                             ...                        
528    Drama Action History Thriller paris assassinat...
529    Action Drama War u.s. army nigeria president H...
530    Action Comedy Thriller Romance assassin Perfec...
531    Comedy Action Adventure spy cold war remake ba...
532    Comedy upper class mother single parent parent...
Length: 533, dtype: object


In [ ]:
#converting the text data to feature vector

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14943 stored elements and shape (533, 4243)>
  Coords	Values
  (0, 31)	0.060394421689375886
  (0, 48)	0.061050160619995616
  (0, 1279)	0.08722004770771886
  (0, 3321)	0.08564885193465269
  (0, 1314)	0.08564885193465269
  (0, 890)	0.24491017404312235
  (0, 723)	0.2298352109827025
  (0, 1440)	0.15252275227722106
  (0, 3520)	0.3050455045544421
  (0, 4044)	0.13543760055412712
  (0, 764)	0.2298352109827025
  (0, 3491)	0.19833315645054664
  (0, 1204)	0.24491017404312235
  (0, 3739)	0.07065945722866872
  (0, 4178)	0.1230677890743829
  (0, 2719)	0.10550573625247181
  (0, 2809)	0.2298352109827025
  (0, 3268)	0.15410509463516792
  (0, 4182)	0.21084300362725736
  (0, 4235)	0.21084300362725736
  (0, 3262)	0.21084300362725736
  (0, 3438)	0.24491017404312235
  (0, 4076)	0.2298352109827025
  (0, 3576)	0.18898942835378885
  (0, 2139)	0.24491017404312235
  :	:
  (532, 1274)	0.07935331879222897
  (532, 1659)	0.14103687058135203
  (532, 3086)	

Cosine Similarity

In [ ]:
#getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)

(533, 533)


In [ ]:
#getting the movie name from user

movie_name = input('Enter your favourite movie name:')

Enter your favourite movie name:iron man


In [ ]:
#creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#finding the close match for thr movie name given by the user

find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [ ]:
#finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [ ]:
#getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.021314313094813846)), (1, np.float64(0.04735711955558237)), (2, np.float64(0.007791737379751846)), (3, np.float64(0.003997677353250027)), (4, np.float64(0.020208677471289176)), (5, np.float64(0.007266820654880758)), (6, np.float64(0.0902048763513027)), (7, np.float64(0.20123954881191936)), (8, np.float64(0.003951034059364093)), (9, np.float64(0.052979958792365875)), (10, np.float64(0.05351719406517293)), (11, np.float64(0.006171478423257472)), (12, np.float64(0.007772079021351156)), (13, np.float64(0.006615208130106753)), (14, np.float64(0.06738298347054403)), (15, np.float64(0.003597767265717092)), (16, np.float64(0.19429425979491927)), (17, np.float64(0.006947857587526831)), (18, np.float64(0.03753430628411818)), (19, np.float64(0.07487405103137468)), (20, np.float64(0.04915422626180025)), (21, np.float64(0.006171762010668024)), (22, np.float64(0.003467772548188023)), (23, np.float64(0.00322806828276093)), (24, np.float64(0.006123499507384207)), (25, np.float64(0.0)

In [ ]:
len(similarity_score)

533

In [ ]:
#sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(68, np.float64(1.0000000000000002)), (79, np.float64(0.361079505592804)), (31, np.float64(0.28182882485652894)), (7, np.float64(0.20123954881191936)), (16, np.float64(0.19429425979491927)), (26, np.float64(0.17400898514369095)), (85, np.float64(0.159189526715827)), (182, np.float64(0.13791284498856973)), (511, np.float64(0.12207885893729999)), (353, np.float64(0.12028281039277251)), (131, np.float64(0.11399660939205405)), (138, np.float64(0.10797835075936249)), (46, np.float64(0.10700057241116256)), (64, np.float64(0.1037512797559809)), (203, np.float64(0.10285056862977199)), (307, np.float64(0.0995247815636352)), (174, np.float64(0.09663328879617585)), (169, np.float64(0.0960861764711727)), (101, np.float64(0.09492181284636889)), (94, np.float64(0.09415118571177704)), (126, np.float64(0.09411971619219889)), (33, np.float64(0.09059045269144067)), (6, np.float64(0.0902048763513027)), (205, np.float64(0.08735030770351083)), (356, np.float64(0.08566104702396266)), (198, np.float64(0.085

In [ ]:
#print the name of similar movies based on the index

print('Movies suggested for you:\n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i+=1

Movies suggested for you:

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Tropic Thunder
11 . G-Force
12 . The Last Airbender
13 . X-Men: Days of Future Past
14 . X-Men: Apocalypse
15 . X2
16 . The Expendables 3
17 . The Incredible Hulk
18 . Captain America: The First Avenger
19 . X-Men: First Class
20 . Guardians of the Galaxy
21 . Thor: The Dark World
22 . X-Men: The Last Stand
23 . Tangled
24 . Sherlock Holmes: A Game of Shadows
25 . Sherlock Holmes
26 . R.I.P.D.
27 . You Don't Mess with the Zohan
28 . TRON: Legacy
29 . Surf's Up


Movie Recommendation system


In [ ]:
movie_name = input('Enter your favourite movie name:')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse=True)

print('Movies suggested for you:\n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i+=1




Enter your favourite movie name:avenger
Movies suggested for you:

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Captain America: Civil War
5 . Iron Man 2
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . Thor
10 . Captain America: The First Avenger
11 . Ant-Man
12 . X-Men: Apocalypse
13 . X-Men: The Last Stand
14 . X2
15 . Iron Man
16 . X-Men: Days of Future Past
17 . Iron Man 3
18 . The Amazing Spider-Man 2
19 . Man of Steel
20 . Zodiac
21 . Batman v Superman: Dawn of Justice
22 . Teenage Mutant Ninja Turtles: Out of the Shadows
23 . X-Men: First Class
24 . The Jungle Book
25 . Guardians of the Galaxy
26 . Sherlock Holmes
27 . The Island
28 . Watchmen
29 . Fantastic Four


In [ ]:
# Re-loading the data and performing necessary preprocessing steps
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#loading the data from the csv file to pandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

#selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']

#replacing the missing values with null values
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

#combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

#converting the text data to feature vector
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

#getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

#creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()

# Recommendation logic
movie_name = input('Enter your favourite movie name:')

find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)

if find_close_match: # Check if a close match was found
  close_match = find_close_match[0]

  index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

  similarity_score = list(enumerate(similarity[index_of_the_movie]))

  sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse=True)

  print('Movies suggested for you:\n')

  i = 1

  for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
      print(i,'.',title_from_index)
      i+=1
else:
  print("Sorry, no close match found for the movie you entered.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/movies.csv'